# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [205]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
from numpy.random import normal
import numpy as np
from math import ceil
import random
from copy import deepcopy

## The *Nim* and *Nimply* classes

In [206]:
Nimply = namedtuple("Nimply", "row, num_objects")
sigma = 0.3


class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [207]:

def pure_random(state: Nim,weights=None) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)
    
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    #print(tmp)
    xor = tmp.sum(axis=0) % 2
    #print(xor)
    return int("".join(str(_) for _ in xor), base=2)


def gabriele(state: Nim,weights=None) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def player1(state:Nim,weights=None)->Nimply:
    """Pick always the maximum possible number of the biggest row""" 
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (m[1])))

def player2(state:Nim,weights=None)->Nimply:
    """Pick always the 1 number of the biggest row""" 
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    row=max(possible_moves, key=lambda m: (m[1]))
    return Nimply(row[0],1)


def player3(state:Nim,weights=None)->Nimply:
    """Pick always the 1 number of the lowest row""" 
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    row=min(possible_moves, key=lambda m: (m[1]))
    return Nimply(row[0],1)

def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked

def optimal(state: Nim) -> Nimply: 
    analysis = analize(state) 
    logging.debug(f"analysis:\n{pformat(analysis)}") 
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0] 
    if not spicy_moves: 
        spicy_moves = list(analysis["possible_moves"].keys()) 
    ply = random.choice(spicy_moves) 
    return ply




In [208]:
def match(strategy0,strategy1,weights)->bool:
    #logging.getLogger().setLevel(logging.INFO)

    #strategy = (gabriele, move)
    strategy=(strategy0, strategy1)
    nim = Nim(4)
    #logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim,weights)
        #print(f"ply: player {player} plays {ply}")
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
    #logging.info(f"status: Player {player} won!")
    return player

# TASK 2.1


Trying to play games with different strategies, we realized that the optimal strategy proposed was not the best since it sometime loses against gabriele or against the random strategy. 

In the example below player 0 (optimal) is playing against player 1 (random):

INFO:root:init : <1 3 5 7 9> 

INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7) 

INFO:root:status: <1 3 5 0 9> 

INFO:root:ply: player 1 plays Nimply(row=4, num_objects=8) 

INFO:root:status: <1 3 5 0 1> 

INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4) 

INFO:root:status: <1 3 1 0 1> 

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1) 

INFO:root:status: <0 3 1 0 1> 

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1) 

INFO:root:status: <0 3 1 0 0> 

INFO:root:ply: player 1 plays Nimply(row=1, num_objects=3) 

INFO:root:status: <0 0 1 0 0> 

INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1) 

INFO:root:status: <0 0 0 0 0> 

INFO:root:status: Player 1 won!

Therefore we implemented the following strategy that mantains the xor = 0 since we arrive to a stopping point where the strategy becomes fixed. 
In fact, when we have just one row with more than one element we should change the nim sum strategy to mantain an odd number of rows with 1 element.

In [ ]:
def expert_agent(state: Nim,weights=None) -> Nimply:
    """Follow the min sum !=0 expect for the last moves""" 
    analysis = analize(state)
    
    one_stick_row=state.rows.count(1)
    more_one_stick_rows = len(state.rows)-one_stick_row-state.rows.count(0)
    

    if more_one_stick_rows==1:
        #print("if more_one")
       
        element=0
        for r in state.rows:
            if r>1:
                element=r
        row_index=state.rows.index(element)
        #ex INFO:root:status: <0 2 1 1 1>
        #if the number of rows with 1 is even leave 1 stick
        #otherwise leave 0
        if one_stick_row % 2==0:
            #print("if one stick")
            return Nimply(row_index,element-1)
        else:
            return Nimply(row_index,element)  
    

    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    #print(analysis["possible_moves"].values())
    
    
    ply = random.choice(spicy_moves)
    return ply


In [209]:
num_eras=50
strategies=[pure_random, gabriele, optimal, player1, player2,player3,expert_agent]
#strategies=[pure_random, gabriele, optimal, expert_agent, player2,player3]

def evolving_strategy(state:Nim,w)->Nimply:
    moves=[]
    dic={}   
    voting={}
    for s in strategies:
        #I collect the suggested moves for each strategy
        moves.append(s(state))

    for i in range(len(moves)) :
    
        if moves[i] in dic.keys():
            dic[moves[i]].append(w[i])
        else:
            dic[moves[i]]=[w[i]]

    for key, value in dic.items():
    
        #for each move i count how many strategies suggested it 
        #and sum the relative weights
        voting[key]=sum(value)+len(value)
    
    #print("dict", dic)
    #print("voting",voting)
    #print(voting)
    max_key = max(voting, key=voting.get)
    #print(max_key)
    return max_key

def fitness(w):
    
    counter=0
    #index=w.index(max(w))
    #print(index)
    for era in range(num_eras):
        
        if(era<num_eras/2):
            if match(evolving_strategy,expert_agent,w)==0:
                counter+=1
            
            if match(evolving_strategy,pure_random,w)==0:
                counter+=1
                      
            if match(evolving_strategy,gabriele,w)==0:
                counter+=1 
           
    
        else:
            if match(expert_agent,evolving_strategy,w)==1:
                counter+=1
            if match(pure_random,evolving_strategy,w)==1:
                counter+=1
            
            if match(gabriele,evolving_strategy,w)==1:
                counter+=1 
            
    #print("games won ", counter)
    return counter

    

## (1+λ) Strategy

In [210]:
num_iterations=100
l=5
#initialize the weights
weights=[]
for _ in range(len(strategies)):
    weights.append(random.random())

print("weights",weights)

index=weights.index(max(weights))

print("max index", index)

prev_won=0
new_won=0
improvements=0
for it in range(num_iterations):
    print("iteration n: ", it)
    counter=fitness(weights)
    
    new_counters=[]
    new_counters={}
    for _ in range(l):
        new_weights=[]
        for i in range(len(weights)):
            #tweak the weights
            new_weights.append(weights[i]+normal(0.0,sigma))
        #print(new_weights)
        #new_counters[new_weights]=fitness(new_weights)
        #print("fitness new weights")
        new_fitness=fitness(new_weights)
        if(new_fitness>counter):
            improvements+=1
            weights=new_weights
            counter=new_fitness
    
    #iterations after which check
    check_it=num_iterations/10
    if (it+1)%check_it==0:
        if improvements/check_it>1/5:
            sigma*=1.1
        else:
           sigma/=1.1
        improvements=0
    
           
    print("weights", weights)
    #print("won matches", counter)

index=weights.index(max(weights))

print("max index", index)


weights [0.9557991908385575, 0.03711551276018199, 0.5212821375184028, 0.7129285683314902, 0.6030039014868817, 0.6233121091168069, 0.13052208143731436]
max index 0
iteration n:  0
weights [0.6441674881365604, 0.38810495590393285, 0.8732349812681686, 0.7845366904666992, 0.9940742717629282, 0.6900018382621883, -0.12165605873113733]
iteration n:  1
weights [0.6441674881365604, 0.38810495590393285, 0.8732349812681686, 0.7845366904666992, 0.9940742717629282, 0.6900018382621883, -0.12165605873113733]
iteration n:  2
weights [-0.10479104465844004, 0.002037347498967855, -0.12338151338641612, 1.1252862993134722, 1.7666372783063515, 0.8142361270313413, -0.07315875896405546]
iteration n:  3
weights [-0.2552019527612608, -0.1496101740716697, -0.4548647381927398, 1.2126798020863003, 1.5813450892769845, 0.4372382785280301, 0.2248684142574227]
iteration n:  4
weights [-0.08431942131459677, -0.3482845084257554, -0.56315953363384, 1.3498237253424437, 1.656650029886623, 0.5970864843244786, 0.367197026709